In [ ]:
# 参考 https://blog.langchain.dev/enhancing-rag-based-applications-accuracy-by-constructing-and-leveraging-knowledge-graphs/

必要パッケージのインストール

In [ ]:
%%capture
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs

必要パッケージのインポート

In [ ]:
import os
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough, ConfigurableField
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

try:
    import google.colab
    from google.colab import output
    output.enable_custom_widget_manager()
except:
    pass

APIキーなどの読み込み

In [ ]:
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass(prompt = 'OpenAIのAPIキーを入力してください')
os.environ["NEO4J_URI"] = getpass.getpass(prompt = 'NEO4JのURIを入力してください')
os.environ["NEO4J_USERNAME"] = getpass.getpass(prompt = 'NEO4JのUSERNAMEを入力してください')
os.environ["NEO4J_PASSWORD"] = getpass.getpass(prompt = 'NEO4Jのパスワードを入力してください')

OpenAIのAPIキーを入力してください··········
NEO4JのURIを入力してください··········
NEO4JのUSERNAMEを入力してください··········
NEO4Jのパスワードを入力してください··········


小説[円城塔「鉄道模型の夜」](https://www.aozora.gr.jp/cards/001916/files/61171_74826.html)を読み込む

In [ ]:
raw_documents = TextLoader('鉄道模型の夜.txt').load()
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=125)
documents = text_splitter.split_documents(raw_documents)

LLM（ChatGPT）でドキュメントをグラフ化する

In [ ]:
llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125")
llm_transformer = LLMGraphTransformer(llm=llm)
graph_documents = llm_transformer.convert_to_graph_documents(documents)

グラフ化したドキュメント情報の確認

In [ ]:
graph_documents

[GraphDocument(nodes=[Node(id='彼', type='Person', properties={}), Node(id='箱庭', type='Hobby', properties={}), Node(id='鉄道', type='Interest', properties={}), Node(id='家', type='Miniature', properties={}), Node(id='押絵', type='Hobby', properties={}), Node(id='風呂敷', type='Traveling', properties={}), Node(id='旅', type='Traveling', properties={}), Node(id='宇宙', type='Space', properties={}), Node(id='気風', type='Trait', properties={}), Node(id='風景', type='Scenery', properties={}), Node(id='空気穴', type='Requirement', properties={}), Node(id='道', type='Path', properties={})], relationships=[Relationship(source=Node(id='彼', type='Person', properties={}), target=Node(id='箱庭', type='Hobby', properties={}), type='INTEREST', properties={}), Relationship(source=Node(id='彼', type='Person', properties={}), target=Node(id='鉄道', type='Interest', properties={}), type='INTEREST', properties={}), Relationship(source=Node(id='箱庭', type='Hobby', properties={}), target=Node(id='家', type='Miniature', properties={

In [ ]:
graph = Neo4jGraph()

In [ ]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [ ]:
# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    #display(widget)
    return widget

グラフの表示

In [ ]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

ハイブリッド検索の準備

In [ ]:
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [ ]:
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

[]

In [ ]:
# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

entity_chain = prompt | llm.with_structured_output(Entities)

In [ ]:
def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and appending a
    similarity threshold (~2 changed characters) to each word, then combines
    them using the AND operator. Useful for mapping entities from user questions
    to database values, and allows for some misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

# Fulltext index query
def structured_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:20})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 1000
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [ ]:
print(structured_retriever("レモンと関わりがあるエンティティを知りたい"))

レモン - 関連 -> 爆弾
レモン - 関連 -> 南
レモン - なる -> 木
レモン - される -> 輪切り
レモン - 中に沈める -> ソーダ
レモン - 中に沈める -> 蜂蜜
レモン - 細切り -> 皮
レモン - 刃を当てる -> 皮
レモン - USED_WITH -> オリーブ油
レモン - BECOMES -> 木
レモン - PLANTS_NEAR -> 対岸の島
レモン - PLANTS_NEAR -> 駅
レモン - DECORATE -> 対岸
彼 - 求める -> レモン
島 - HAS_PLANT -> レモン
爆弾 - 関連付ける -> レモン
箱庭 - 必要 -> レモン
一個 - 過大 -> レモン
少年 - HOLDING -> レモン
自転車乗り - SOUGHT -> レモン
島 - GROWING -> レモン
彼 - PLANTS_NEAR -> レモン
対岸 - DECORATED_WITH -> レモン
少年 - MATERIAL_USED -> レジン
レンズ - 強化 -> 印象
レンズ - 試す -> 広角
カメラ - 構える -> レンズ
少年 - VIEWING_AS -> クレーン
志賀直哉 - BUYING -> パン
志賀直哉 - PURCHASED -> パン
パン屋航路 - HAS_SIGN -> パン屋
作家 - COLLEAGUE -> パン屋
パン屋航路 - HAS_SIGN -> パン屋


In [ ]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
    {structured_data}
    Unstructured data:
    {"#Document ". join(unstructured_data)}
    """
    # print(final_data)
    return final_data

In [ ]:
_search_query = RunnableLambda(lambda x: x["question"])

In [ ]:
template = """あなたは優秀なAIです。下記のコンテキストを利用してユーザーの質問に丁寧に答えてください。
必ず文脈からわかる情報のみを使用して回答を生成してください。
{context}

ユーザーの質問: {question}"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke({"question": "トマトスキヤキとは何？"})

Search query: トマトスキヤキとは何？


'回答: トマトスキヤキは、尾道と関連がある料理です。林芙美子のトマトのすき焼きというレシピが知られており、トマトを薄く輪切りにして炒め、肉や醤油、甘味料と一緒に調理する料理です。また、尾道でトマトスキヤキを食べたわけではなく、尾道ラーメンやオコゼの唐揚げを試したという記述があります。'

In [ ]:
chain.invoke({"question": "トマトスキヤキのレシピは？"})

Search query: トマトスキヤキのレシピは？


'トマトスキヤキのレシピは、薄く輪切りにしたトマトをバターかラードで炒め、火が通ったら肉を載せ、醤油と甘味料を加えるというものです。また、別のレシピでは、輪切りにした玉葱を炒めておいて、トマトを投入し、割り下を注ぎ、バジルの葉を入れ、牛肉を広げて溶き卵につけて食べるという方法もあります。'

In [ ]:
chain.invoke({"question": "取材先はどこ？"})

Search query: 取材先はどこ？


'取材先は尾道です。'